# IT1244 Project

## Import Libraries

In [3]:
import re as re
import heapq as heapq
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MultiLabelBinarizer, Normalizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import random as random
from sklearn.preprocessing import StandardScaler
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.decomposition import PCA
#import torch as torch
#from transformers import AutoModel
#from transformers import AutoTokenizer, BertTokenizerFast
#from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, ComplementNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

## Data Importing

In [2]:
tweets = pd.read_csv("../Data/Raw/dataset.csv", header=None)
tweets.head()

,0,1
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [3]:
Bert_Encoded_Text = pd.read_csv("../Data/Cleaned/BERT_Vectors.csv").iloc[: , 1:]
X = Bert_Encoded_Text
y = tweets[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# Scale the features using StandardScaler
#scaler = MinMaxScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

y_pred = nb_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6607


TF-IDF (not using BERT)

In [1]:
def metrics(y_true, y_guess, table = False, model_name = "", digits=4):
    CM = confusion_matrix(y_true, y_guess)
    TN, FN, TP, FP =  CM[0][0], CM[1][0], CM[1][1], CM[0][1]
    accuracy = np.round((TP+TN)/(TP+TN+FP+FN), digits)
    precision = np.round(TP/(TP+FP), digits)
    recall =  np.round(TP/(TP+FN), digits)
    f1_score =  np.round(2*(precision*recall)/(precision+recall), digits)
    if not table:
        return accuracy, precision, recall, f1_score
    else:
        if not model_name:
            return pd.DataFrame({'accuracy': [accuracy], 'precision': [precision], 'recall': [recall], 'f1_score': [f1_score]})
        else:
            return pd.DataFrame({'model':[model_name], 'accuracy': [accuracy], 'precision': [precision], 'recall': [recall], 'f1_score': [f1_score]})

In [4]:
cleanedtweets = pd.read_csv("../Data/Raw/CleanedTweets.csv")
cleanedtweets["processed_text"] = cleanedtweets["processed_text"].fillna("")
X = cleanedtweets["processed_text"] 
y = cleanedtweets["sentiment"]  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

idf_nb = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Train the model
idf_nb.fit(X_train, y_train)

# Make predictions
y_pred = idf_nb.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"TF-IDF + Naive Bayes Accuracy: {accuracy:.5f}")

confusion_matrix(y_test, y_pred)

TF-IDF + Naive Bayes Accuracy: 0.75980


array([[7900, 2168],
       [2636, 7296]])

In [5]:
metrics(y_test, y_pred, table = True, model_name = "Naive Bayes")

,model,accuracy,precision,recall,f1_score
0,Naive Bayes,0.7598,0.7709,0.7346,0.7523


In [12]:
cleanedtweets = pd.read_csv("../Data/Raw/CleanedTweets.csv")
cleanedtweets["processed_text"] = cleanedtweets["processed_text"].fillna("")
X = cleanedtweets["processed_text"] 
y = cleanedtweets["sentiment"]  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# Create vectorizer
tfidf = TfidfVectorizer()

# Fit and transform training data
X_train_tfidf = tfidf.fit_transform(X_train)

# Transform test data
X_test_tfidf = tfidf.transform(X_test)


In [13]:
X_train_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 577007 stored elements and shape (80000, 42534)>

In [ ]:
model_results = []

for dim in range(25, 250, 25):
    print(f"currently in dimension {dim}")
    n_components = dim  # Desired number of principal components, you can adjust it
    pca = PCA(n_components=n_components)
    tdidf_vectors_pca = pca.fit_transform(X_train_tfidf)

    X = tdidf_vectors_pca
    y = tweets[0]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    k_values = [i for i in range(1, 50, 2)]
    scores = []

    for k in k_values:                
        knn = KNeighborsClassifier(n_neighbors=k)
        score = cross_val_score(knn, X_train, y_train, cv=5)
        scores.append(np.mean(score))

    results = list(map(lambda x: float(x), scores))

    # Compute the differences between consecutive scores
    score_differences = np.diff(results)

    # Find the index where the difference between scores starts to decrease significantly
    # Look for the point where the difference drops below a threshold
    threshold = 0.001  # You can adjust this threshold based on your observations
    elbow_point = np.where(score_differences < threshold)[0][0]  # Index of the "elbow"

    # The optimal k is the k corresponding to this elbow point
    optimal_k_elbow = k_values[elbow_point]

    #optimal k
    knn_pca = KNeighborsClassifier(n_neighbors=optimal_k_elbow)
    knn_pca.fit(X_train, y_train)
    y_pred_pca = (knn_pca.predict(X_test) > 0.5)
    mi = metrics(y_test, y_pred_pca, table = True, model_name=dim)
    model_results.append(mi)

final_results = pd.concat(model_results).reset_index(drop=True)

BoW (not using BERT)

In [7]:
cleanedtweets = pd.read_csv("../Data/Raw/CleanedTweets.csv")
cleanedtweets["processed_text"] = cleanedtweets["processed_text"].fillna("")
X = cleanedtweets["processed_text"] 
y = cleanedtweets["sentiment"]  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# BoW + Naïve Bayes pipeline
bow_nb = make_pipeline(CountVectorizer(), KNeighborsClassifier())

# Train the model
bow_nb.fit(X_train, y_train)

# Make predictions
y_pred = bow_nb.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"BoW + Naïve Bayes Accuracy: {accuracy:.5f}")

confusion_matrix(y_test, y_pred)

KeyboardInterrupt: 